### Some packages 


In [9]:
import numpy as np 
import matplotlib.pyplot as plt 
from keras.models import Sequential 
from keras.layers import Dense, Dropout
# dropout is supposed to avoid a bit of overfitting becasue it allows the NN not depending on a particular node removing for each iteration one of them
from keras.optimizers import SGD 
import time 
import math 

## Data (strings) and splitting

In [10]:
fname = 'DATA/sequences12.csv'
sx, sy = np.loadtxt(fname, delimiter=',',
                    usecols=(0,1),
                    unpack=True, 
                    dtype = str)
N = len(sy)
print(N)
# essendo tante labels quanti sono i dati 

Ls = len(sx[0])
print(Ls)
# mentre Ls altro non è che la lunghezza di ogni array estratto dal file csv presa singolarmente

print(sx[0], sy[0])

perc_train = 0.8
N_train = int(N* perc_train)
N_test = N - N_train 
print(f'\ndata: {N}\ntrain: {N_train}\ntest: {N_test}')

10000
12
TAGGCGTCGATG 0

data: 10000
train: 8000
test: 2000


In [18]:
# it is necessary to transform our sequences of letters into numbers, converting every piece of the data into a sequence of bits depending on a dictionary of characters we are going to make 
Q = ['A','C','G','T']
Nc = 4
onehc = {Q[i]:i for i in range(Nc)}
print(onehc)
# the dictionary is created like in a list comprehension 
# con il one hot encoding stiamo convertendo ogni lettera in una sequanza di bits: 
# A : 1000, C : 0100, G : 0010, T : 0001
# mentre la scelta di attribuire a ciascuna lettera un numero reale potrebbe essere arbitraria e far nascere qualche tipo di preferenza nella classificazione all'interno del neural network, utilizzando sequenze di 4 bits evitiamo questo problema perchè sono tutte e quattro equiprobabili in un certo senso


{'A': 0, 'C': 1, 'G': 2, 'T': 3}


### Data conversion 


In [19]:
y = sy.astype(int) # casting everything int
# convertire le labels è semplice perchè sono N interi 0 oppure 1

L = Ls*Nc
print(L)
# perchè ogni lettera è tradotta in quattro bits

x = np.zeros((N,L))
print(x[0])

for n in range(N): 
# quindi per tutte le righe 
    for i in range(Ls): 
# per ogni lettera 
        x[n][i*4 + onehc[sx[n][i]]] = 1 
# 4*i perchè per ogni step di una lettera abbiamo uno spostamento di quattro bits nella sequenza convertita.
# Vediamo che nel dizionario il valore che accompagna ciascuna carattere corrisponde alla posizione del numero uno nella sequenza di 4 bits, quindi scegliendo una riga x[n] cambiamo ogni quattro bits il valore alla posizione onehc[sx[n][i]] e lo poniamo uguale a uno. (con onehc[] scegliamo il valore della chiave corrispondente alla lettera che andremo a mettere tra parentesi essendo onehc un dizionario)

print(sx[0])
print(x[0])

48
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
TAGGCGTCGATG
[0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]


### Split train / test-validation

In [5]:
x_train, y_train = x[:N_train], y[:N_train]
x_test, y_test = x[:N_test], y[:N_test]

print(y_train.sum() / N_train)
print(y_test.sum() / N_test)

0.314125
0.3165


### Definition of model in Keras


In [6]:
np.random.seed(123)

model = Sequential()

#I choose narrower dimensions of layers in my model (that's my choice)
#also activation is my choice
model.add(Dense(L,input_shape=(L,),activation="relu")) #set input layer of shape (1,0)
model.add(Dense(L/2,activation="relu"))
model.add(Dense(L/4,activation="relu"))
model.add(Dropout(0.2)) #drop non active nodes in the last layer(?)

#now I change the activation function because I want a prediciton as a probability of being 1
model.add(Dense(1,activation="sigmoid")) #OUTPUT


print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 48)                2352      
_________________________________________________________________
dense_1 (Dense)              (None, 24)                1176      
_________________________________________________________________
dense_2 (Dense)              (None, 12)                300       
_________________________________________________________________
dropout (Dropout)            (None, 12)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 13        
Total params: 3,841
Trainable params: 3,841
Non-trainable params: 0
_________________________________________________________________
None


### Optimization


In [8]:
opt1 = SGD(learning_rate=0.1,
            momentum=0.9,
            nesterov=True) 

model.compile(loss="binary_crossentropy", #check what it is
              optimizer=opt1, #"adam"
              metrics=['accuracy']) 

fit = model.fit(x_train, y_train, 
               epochs=30, batch_size=50,
               validation_data=(x_test,y_test),
                shuffle=True)

Epoch 1/30
160/160 [==============================] - 1s 2ms/step - loss: 0.5513 - accuracy: 0.7111 - val_loss: 0.3994 - val_accuracy: 0.8080
Epoch 2/30
160/160 [==============================] - 0s 903us/step - loss: 0.4134 - accuracy: 0.8029 - val_loss: 0.3332 - val_accuracy: 0.8455
Epoch 3/30
160/160 [==============================] - 0s 888us/step - loss: 0.3063 - accuracy: 0.8692 - val_loss: 0.2004 - val_accuracy: 0.9170
Epoch 4/30
160/160 [==============================] - 0s 846us/step - loss: 0.2187 - accuracy: 0.9102 - val_loss: 0.1622 - val_accuracy: 0.9270
Epoch 5/30
160/160 [==============================] - 0s 878us/step - loss: 0.1770 - accuracy: 0.9302 - val_loss: 0.1283 - val_accuracy: 0.9515
Epoch 6/30
160/160 [==============================] - 0s 878us/step - loss: 0.1372 - accuracy: 0.9482 - val_loss: 0.0965 - val_accuracy: 0.9620
Epoch 7/30
160/160 [==============================] - 0s 897us/step - loss: 0.1059 - accuracy: 0.9600 - val_loss: 0.1137 - val_accuracy: 0